In [1]:
import pandas as pd
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('sparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
print(sc)
numeric_val = sc.parallelize([1,2,3,4])
numeric_val.map(lambda x:x*x*x).collect()
sc.stop
import mllib

<SparkContext master=local appName=sparkApp>


In [2]:
#import module
from pyspark.ml.recommendation import ALS

#create session
appName = "Recommender system in Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
# read data
data_orin = spark.read.csv('./data/movieRating.csv', inferSchema=True, header=True).select("UserId", "MovieId", "Rating")
# split training set and test set
splits = data_orin.randomSplit([0.8, 0.2], seed=10)
train = splits[0].withColumnRenamed("Rating", "Label")
test = splits[1].withColumnRenamed("Rating", "trueLabel")
# aaa.printSchema()
train.show(5)
test.show(5)
train_rows = train.count()
test_rows = test.count()
print ("number of training data rows:", train_rows, 
       ", number of testing data rows:", test_rows)

+------+-------+-----+
|UserId|MovieId|Label|
+------+-------+-----+
|     1|     11|    5|
|     1|     85|    3|
|     1|    249|    4|
|     1|    260|    5|
|     1|    265|    4|
+------+-------+-----+
only showing top 5 rows

+------+-------+---------+
|UserId|MovieId|trueLabel|
+------+-------+---------+
|     1|     36|        5|
|     1|    150|        5|
|     1|    261|        4|
|     1|    356|        5|
|     1|    531|        5|
+------+-------+---------+
only showing top 5 rows

number of training data rows: 719625 , number of testing data rows: 180248


In [4]:
# define ALS (Alternating Least Square) as recommendation model
als = ALS(maxIter=19, regParam=0.01, userCol="UserId", 
          itemCol="MovieId", ratingCol="Label")
#train ALS model
model = als.fit(train)
print("Training is done!")

Training is done!


In [5]:
# generate prediction
prediction = model.transform(test)
print("testing is done!")

testing is done!


In [6]:
#import RegressionEvaluator to calculate MAE
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(prediction)
print ("MAE:", mae)

MAE: nan


In [12]:
# 因為MAE=nan，判斷原本的prediction中有空值，需刪除
a = prediction.count()
print("number of original data rows: ", a)
#drop rows with any missing data
cleanPred = prediction.dropna(how="any", subset=["prediction"])
b = cleanPred.count()
print("number of rows after dropping data with missing value: ", b)
print("number of missing data: ", a-b)

number of original data rows:  180248
number of rows after dropping data with missing value:  180226
number of missing data:  22


In [13]:
# generate MAE
mae = evaluator.evaluate(cleanPred)
print ("mae:", mae)

mae: 0.6972219239848941


In [14]:
# write prediction result into csv file
pred = prediction.toPandas()
pred.to_csv('./result/prediction.csv')

In [15]:
# Root Mean Square Error (RMSE): 0.9012930533111924

In [16]:
# can't calculate regression model's accuracy !!!!!!!! 